In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [2]:
bigM = 1000000

In [3]:
# Load data
locations_df = pd.read_csv('locations.csv')
order_list_df = pd.read_excel('order_list_1.xlsx')
travel_matrix_df = pd.read_csv('travel_matrix.csv')
trucks_df = pd.read_csv('trucks.csv')

In [4]:
Q = sorted(list(set(trucks_df['truck_max_weight'])))

In [5]:
Q = [Q[0]]*10 + [Q[1]]*8 + [Q[2]]*10 + [Q[3]]*4 + [Q[4]]*5

In [6]:
dest1 = list(set(order_list_df['Destination Code']))[:15]

In [7]:
dest = [str(i) for i in dest1]

In [8]:
order_list_df = order_list_df[order_list_df['Destination Code'].isin(dest1)]

In [9]:
order_list_df1 = order_list_df.sort_values(by = 'Destination Code').groupby('Destination Code').sum().reset_index()

In [10]:
locations_df = locations_df[locations_df['location_code'].isin(dest + ['A123'])]

In [11]:
# Convert loading/unloading windows to minutes with explicit format
locations_df['start_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.minute
locations_df['end_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.minute

In [12]:
customers = locations_df.sort_values(by = 'location_code').iloc[:len(order_list_df1),:]

In [13]:
locations_df2 = locations_df.sort_values(by = 'location_code')

In [14]:
locations_df2

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
36,10208768,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
65,10208776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
45,10212373,['3 tonner Box'],8:00,22:00,480,1320
16,10218045,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
74,10310757,['3 tonner Box'],8:00,22:00,480,1320
12,10364446,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
55,10366979,['3 tonner Box'],8:00,22:00,480,1320
101,11950159,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
11,11956776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
102,11957296,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [15]:
cap_df = dict(zip(trucks_df['truck_type'], trucks_df['truck_max_weight']))

In [16]:
max_veh_access = []
for i in locations_df2.index:
    max_veh_access.append(cap_df[eval(locations_df2['trucks_allowed'][i])[-1]])

In [17]:
max_veh_access = max_veh_access[len(order_list_df1):] + max_veh_access[:len(order_list_df1)]

In [18]:
max_veh_access

[17000,
 7000,
 7000,
 2800,
 9200,
 2800,
 7000,
 2800,
 7000,
 7000,
 7000,
 2800,
 2800,
 17000,
 17000,
 9200]

In [19]:
depot = locations_df.sort_values(by = 'location_code').iloc[len(order_list_df1):,:]

In [20]:
Nodes = pd.concat([depot,customers], ignore_index = True)

In [21]:
Nodes

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
0,A123,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,18:00,480,1080
1,10208768,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
2,10208776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
3,10212373,['3 tonner Box'],8:00,22:00,480,1320
4,10218045,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
5,10310757,['3 tonner Box'],8:00,22:00,480,1320
6,10364446,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
7,10366979,['3 tonner Box'],8:00,22:00,480,1320
8,11950159,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
9,11956776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [22]:
vehicles = [ k for k in range(0,len(Q))]
customers = [ i for i in range(1,len(Nodes))]
nodes = [ i for i in range(0, len(Nodes))]

In [23]:
sum(Q)

257400

In [24]:
demand = [0] + list(order_list_df1['Total Weight'])

In [25]:
sum(demand)

68918.0

In [26]:
start_time = list(Nodes['start_minutes'])
finish_time = list(Nodes['end_minutes'])

In [27]:
# Constants
service_time_customer = 20  
service_time_depot = 60 

In [28]:
dest2 = ['A123'] + sorted(dest)

In [29]:
dest3 = {} 
for i in range(len(dest2)):
    dest3[dest2[i]] = i

In [30]:
dest3

{'A123': 0,
 '10208768': 1,
 '10208776': 2,
 '10212373': 3,
 '10218045': 4,
 '10310757': 5,
 '10364446': 6,
 '10366979': 7,
 '11950159': 8,
 '11956776': 9,
 '11957296': 10,
 '12219936': 11,
 '12231175': 12,
 '12474916': 13,
 '12779571': 14,
 '13015040': 15}

In [31]:
travel_matrix_df = travel_matrix_df[(travel_matrix_df['source_location_code'].isin(dest + ['A123'])) & (travel_matrix_df['destination_location_code'].isin(dest + ['A123']))]

In [32]:
travel_matrix_df['mapped_source'] = travel_matrix_df['source_location_code'].map(dest3)
travel_matrix_df['mapped_destination'] = travel_matrix_df['destination_location_code'].map(dest3)

In [33]:
dist_matrix = {}
time_matrix = {}
for i in travel_matrix_df.index:
    dist_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_distance_in_km'][i]
    time_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_time_in_min'][i]

In [34]:
start_time

[480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480]

In [35]:
finish_time

[1080,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320]

In [36]:
nodes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [37]:
customers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [38]:
vehicles

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36]

In [39]:
#Decision Variable

my_model=gp.Model('CVRPTW')

xijk=my_model.addVars(nodes, nodes, vehicles, vtype=GRB.BINARY, name='xijk')
sik = my_model.addVars(nodes, vehicles, vtype = GRB.CONTINUOUS, name = 'sik',lb = 0)
q = my_model.addVars(customers, vehicles, vtype = GRB.CONTINUOUS, name = 'qjk', lb = 0)
I = my_model.addVars(vehicles, vtype = GRB.BINARY, name = 'I')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-10


In [40]:
#Objective function
obj_fn = (gp.quicksum(dist_matrix[i,j]* gp.quicksum(xijk[i,j,k] for k in vehicles) for i in nodes for j in nodes) +
          gp.quicksum(I[k] for k in vehicles)
         )

my_model.setObjective(obj_fn, GRB.MINIMIZE)


In [41]:
#Source to sink constraints
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers)<=1 for k in vehicles);

In [42]:
my_model.addConstrs(gp.quicksum(xijk[i,0,k] for i in customers)<=1 for k in vehicles);

In [43]:
# my_model.addConstr(gp.quicksum(xijk[0,j,k] for j in customers for k in vehicles)<=10);

In [44]:
my_model.addConstrs(gp.quicksum(xijk[i,j,k] for i in nodes)- gp.quicksum(xijk[j,i,k] for i in nodes)==0 
                    for j in nodes for k in vehicles);

In [45]:
#capacity constraints
my_model.addConstrs(gp.quicksum(q[j,k] for k in vehicles) == demand[j] 
                    for j in customers); 
my_model.addConstrs(gp.quicksum(q[j,k] for j in customers) <= Q[k] for k in vehicles);

In [46]:
my_model.addConstrs(q[j,k] <= bigM * gp.quicksum(xijk[i,j,k] for i in nodes) for j in customers for k in vehicles);

In [47]:
my_model.addConstrs(sik[i,k] + time_matrix[i,j] + service_time_customer
                    - sik[j,k] <= (1-xijk[i,j,k]) *bigM 
                    for i in customers 
                    for j in customers for k in vehicles);

In [48]:
my_model.addConstrs(sik[i,k] <= finish_time[i] for i in nodes for k in vehicles)
                    
my_model.addConstrs(sik[i,k] >= start_time[i] for i in nodes for k in vehicles);                                   

In [49]:
for i in nodes:
    for j in nodes:
        for k in vehicles:
            if Q[k] <= max_veh_access[j] and Q[k] <= max_veh_access[i]:
                my_model.addConstr(xijk[i,j,k] <= 1);
            else:
                my_model.addConstr(xijk[i,j,k] == 0);

In [50]:
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers) <= I[k] for k in vehicles);

In [51]:
my_model.setParam('MIPGap', 0.6)  

Set parameter MIPGap to value 0.6


In [52]:
my_model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 20291 rows, 11211 columns and 64528 nonzeros
Model fingerprint: 0x4b4a8756
Variable types: 1702 continuous, 9509 integer (9509 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 15648 rows and 5991 columns
Presolve time: 0.19s
Presolved: 4643 rows, 5220 columns, 26254 nonzeros
Variable types: 702 continuous, 4518 integer (4518 binary)

Root relaxation: objective 3.141000e+02, 845 iterations, 0.08 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  314.10005    0   38     

In [53]:
# Create a new dictionary to store non-zero values
non_zero_xijk = {}

# Iterate over all keys and values in xijk
for (i, j, k), value in xijk.items():
    # Check if the value is a Gurobi variable
    if hasattr(value, "X"):  # If it's a Gurobi variable, use .X to get the optimized value
        if value.X > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value.X
    else:  # Otherwise, assume it's a numeric value
        if value > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value

# Print the non-zero values
print("Non-zero values of x:")
for (i, j, k), value in non_zero_xijk.items():
    print(f"x_{i}_{j}_{k} = {value}")


Non-zero values of x:
x_0_4_11 = 1.0
x_0_4_12 = 1.0
x_0_4_18 = 1.0
x_0_4_23 = 1.0
x_0_4_24 = 1.0
x_0_4_25 = 1.0
x_0_4_26 = 1.0
x_0_4_27 = 1.0
x_0_7_5 = 1.0
x_0_8_22 = 1.0
x_0_14_30 = 1.0
x_1_6_11 = 1.0
x_1_9_22 = 1.0
x_1_9_26 = 1.0
x_2_0_24 = 1.0
x_2_0_26 = 1.0
x_2_4_22 = 1.0
x_2_11_5 = 1.0
x_3_0_5 = 1.0
x_4_0_22 = 1.0
x_4_1_11 = 1.0
x_4_6_12 = 1.0
x_4_6_18 = 1.0
x_4_6_23 = 1.0
x_4_6_24 = 1.0
x_4_6_25 = 1.0
x_4_8_26 = 1.0
x_4_14_27 = 1.0
x_5_3_5 = 1.0
x_6_0_11 = 1.0
x_6_0_12 = 1.0
x_6_0_18 = 1.0
x_6_1_22 = 1.0
x_6_1_26 = 1.0
x_6_8_25 = 1.0
x_6_9_24 = 1.0
x_6_10_23 = 1.0
x_7_12_5 = 1.0
x_8_0_23 = 1.0
x_8_0_25 = 1.0
x_8_6_22 = 1.0
x_8_6_26 = 1.0
x_9_2_22 = 1.0
x_9_2_24 = 1.0
x_9_13_26 = 1.0
x_10_8_23 = 1.0
x_11_5_5 = 1.0
x_12_2_5 = 1.0
x_13_2_26 = 1.0
x_13_15_30 = 1.0
x_14_0_27 = 1.0
x_14_13_30 = 1.0
x_15_0_30 = 1.0


In [54]:
# Create a dictionary to store routes for each vehicle
vehicle_routes = {}

# Iterate over the non_zero_xijk dictionary to create routes
for (i, j, k), value in non_zero_xijk.items():
    if value == 1.0:
        if k not in vehicle_routes:
            vehicle_routes[k] = []  # Create a list for each vehicle if not already created
        vehicle_routes[k].append((i, j))

# Display the routes for each vehicle
for vehicle, route in vehicle_routes.items():
    print(f"Vehicle {vehicle} Route: {route}")

Vehicle 11 Route: [(0, 4), (1, 6), (4, 1), (6, 0)]
Vehicle 12 Route: [(0, 4), (4, 6), (6, 0)]
Vehicle 18 Route: [(0, 4), (4, 6), (6, 0)]
Vehicle 23 Route: [(0, 4), (4, 6), (6, 10), (8, 0), (10, 8)]
Vehicle 24 Route: [(0, 4), (2, 0), (4, 6), (6, 9), (9, 2)]
Vehicle 25 Route: [(0, 4), (4, 6), (6, 8), (8, 0)]
Vehicle 26 Route: [(0, 4), (1, 9), (2, 0), (4, 8), (6, 1), (8, 6), (9, 13), (13, 2)]
Vehicle 27 Route: [(0, 4), (4, 14), (14, 0)]
Vehicle 5 Route: [(0, 7), (2, 11), (3, 0), (5, 3), (7, 12), (11, 5), (12, 2)]
Vehicle 22 Route: [(0, 8), (1, 9), (2, 4), (4, 0), (6, 1), (8, 6), (9, 2)]
Vehicle 30 Route: [(0, 14), (13, 15), (14, 13), (15, 0)]


In [55]:
max_veh_access

[17000,
 7000,
 7000,
 2800,
 9200,
 2800,
 7000,
 2800,
 7000,
 7000,
 7000,
 2800,
 2800,
 17000,
 17000,
 9200]

In [56]:
non_zero_q = {(i,k):q[i,k].X for i in customers for k in vehicles if q[i,k].X > 0.1}
non_zero_q

{(1, 11): 746.0,
 (2, 5): 611.0,
 (3, 5): 209.0,
 (4, 11): 1122.0,
 (4, 27): 5527.0,
 (5, 5): 310.0,
 (6, 11): 2832.0,
 (6, 12): 4700.0,
 (6, 18): 7000.0,
 (6, 25): 5468.0,
 (7, 5): 304.0,
 (8, 25): 1532.0,
 (9, 22): 7000.0,
 (9, 24): 7000.0,
 (9, 26): 2000.0,
 (10, 23): 7000.0,
 (11, 5): 311.0,
 (12, 5): 84.0,
 (13, 26): 5000.0,
 (13, 30): 8688.0,
 (14, 27): 1254.0,
 (15, 30): 220.0}

In [57]:
demand

[0,
 746.0,
 611.0,
 209.0,
 6649.0,
 310.0,
 20000.0,
 304.0,
 1532.0,
 16000.0,
 7000.0,
 311.0,
 84.0,
 13688.0,
 1254.0,
 220.0]

In [58]:
I

{0: <gurobi.Var I[0] (value -0.0)>,
 1: <gurobi.Var I[1] (value -0.0)>,
 2: <gurobi.Var I[2] (value -0.0)>,
 3: <gurobi.Var I[3] (value -0.0)>,
 4: <gurobi.Var I[4] (value -0.0)>,
 5: <gurobi.Var I[5] (value 1.0)>,
 6: <gurobi.Var I[6] (value -0.0)>,
 7: <gurobi.Var I[7] (value -0.0)>,
 8: <gurobi.Var I[8] (value -0.0)>,
 9: <gurobi.Var I[9] (value 0.0)>,
 10: <gurobi.Var I[10] (value -0.0)>,
 11: <gurobi.Var I[11] (value 1.0)>,
 12: <gurobi.Var I[12] (value 1.0)>,
 13: <gurobi.Var I[13] (value -0.0)>,
 14: <gurobi.Var I[14] (value -0.0)>,
 15: <gurobi.Var I[15] (value -0.0)>,
 16: <gurobi.Var I[16] (value -0.0)>,
 17: <gurobi.Var I[17] (value -0.0)>,
 18: <gurobi.Var I[18] (value 1.0)>,
 19: <gurobi.Var I[19] (value 0.0)>,
 20: <gurobi.Var I[20] (value -0.0)>,
 21: <gurobi.Var I[21] (value -0.0)>,
 22: <gurobi.Var I[22] (value 1.0)>,
 23: <gurobi.Var I[23] (value 1.0)>,
 24: <gurobi.Var I[24] (value 1.0)>,
 25: <gurobi.Var I[25] (value 1.0)>,
 26: <gurobi.Var I[26] (value 1.0)>,
 27: 

In [59]:
#use dest3 for reverse mapping into location codes from index